In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift

from PIL import Image

size = 100, 100

img_names_1 = ["COMP9517_20T2_Lab3_Images/shapes.png", "COMP9517_20T2_Lab3_Images/strawberry.png"]
img_names_2 = ["COMP9517_20T2_Lab3_Images/coins.png", "COMP9517_20T2_Lab3_Images/kiwi.png"]

images_1 = [i for i in img_names_1]
images_2 = [i for i in img_names_2]

def plot_three_images(figure_title, image1, label1,
                      image2, label2, image3, label3):
    fig = plt.figure()
    fig.suptitle(figure_title)

    # Display the first image
    fig.add_subplot(1, 3, 1)
    plt.imshow(image1)
    plt.axis('off')
    plt.title(label1)

    # Display the second image
    fig.add_subplot(1, 3, 2)
    plt.imshow(image2)
    plt.axis('off')
    plt.title(label2)

    # Display the third image
    fig.add_subplot(1, 3, 3)
    plt.imshow(image3)
    plt.axis('off')
    plt.title(label3)

    plt.show()

    
def do_meanshift(img):
    img_mat = np.array(img)[:, :, :3]
    img_r = img_mat[:, :, 0]
    img_g = img_mat[:, :, 1]
    img_b = img_mat[:, :, 2]
    
    shape = img_r.shape
    samples = np.concatenate((img_r.flatten().reshape(-1,1), img_g.flatten().reshape(-1,1), img_b.flatten().reshape(-1,1)), axis=1)
    
    ms_clf = MeanShift(bin_seeding=True)
    ms_labels = ms_clf.fit_predict(samples).reshape(shape)

    return ms_labels

def do_watershed(img):
    img_gray = np.array(img.convert('L'))
    
    img_array = np.array(img_gray)
    distance = ndi.distance_transform_edt(img_array)
    local_maxi = peak_local_max(distance, indices=False, labels=img_array)
    markers = ndi.label(local_maxi)[0] 
    ws_labels = watershed(-distance, markers, mask=img_array)
    return ws_labels, distance
    

In [5]:
for img_path in images_1:
    img = Image.open(img_path)
    img.thumbnail(size)
    
    ms_labels = do_meanshift(img)
    ws_labels, distance = do_watershed(img)
    
    plot_three_images(img_path, img, "Image",
                     ms_labels, "MeanShift Labels",
                     ws_labels, "Watershed Labels")

for img_path in images_2:
    img = Image.open(img_path)
    img.thumbnail(size)
    
    ms_labels = do_meanshift(img)
    ws_labels, distance = do_watershed(img)
    
    plot_three_images(img_path, img, "Image",
                     ms_labels, "MeanShift Labels",
                     ws_labels, "Watershed Labels")

FileNotFoundError: [Errno 2] No such file or directory: 'COMP9517_20T2_Lab3_Images/shapes.png'

In [ ]:
#task 3
def optimization_ws(img):
    img_gray = np.array(img.convert('L'))
    raw_img = np.array(img)[:, :, :3]
    print(img_gray.shape)
    ret, thresh = cv2.threshold(img_gray, 0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    img_array = np.array(img_gray)
    # 噪声去除
    kernel = np.ones((3,3),np.uint8)
    closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 1)
    # 确定背景区域
    sure_bg = cv2.dilate(closing,kernel,iterations=1)
    # 寻找前景区域
    dist_transform = cv2.distanceTransform(sure_bg,cv2.DIST_L2,0)
    
    plt.imshow(dist_transform)
    
    localMax = peak_local_max(dist_transform, indices=False, labels=sure_bg)
    markers = ndi.label(localMax)[0]
    labels = watershed(-dist_transform, markers, mask=img_gray)
    return labels, dist_transform

In [3]:
for img_path in images_2:
    img = Image.open(img_path)
    img.thumbnail(size)
    
    ms_labels = do_meanshift(img)
    ws_labels, distance = optimization_ws(img)
    
    plot_three_images(img_path, img, "Image",
                     ms_labels, "MeanShift Labels",
                     ws_labels, "Watershed Labels")

FileNotFoundError: [Errno 2] No such file or directory: 'COMP9517_20T2_Lab3_Images/coins.png'